In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [19]:
sparkSession = SparkSession.builder.appName("veriMaratonu").getOrCreate()

In [20]:
rawDF = sparkSession.read.csv("ratings.csv",header=True,inferSchema=True)

In [21]:
rawDF.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [22]:
mlDF= rawDF.drop('timestamp')

In [23]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True)

In [24]:
(trainDF, testDF) = mlDF.randomSplit([0.8,0.2])

In [25]:
trainDF.count()
testDF.count()

20168

In [26]:
model = als.fit(trainDF) #verisi fazla df'yi eklemek daha saglikli

In [27]:
predictDF = model.transform(testDF)

In [28]:
predictDF.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     2|     62|   3.0| 3.5516415|
|     2|    185|   3.0|  2.993628|
|     2|    223|   1.0| 3.5583277|
|     2|    248|   3.0|   2.53738|
|     2|    265|   5.0|  3.781381|
|     2|    292|   3.0| 3.0957909|
|     2|    300|   3.0| 3.5611432|
|     2|    350|   4.0| 3.0594378|
|     2|    372|   3.0|  2.612249|
|     2|    377|   3.0|  3.567668|
|     2|    474|   2.0|  3.877839|
|     2|    480|   4.0| 3.5477107|
|     2|    508|   4.0| 3.8319561|
|     2|    537|   4.0| 2.9513986|
|     1|   1029|   3.0| 2.3885026|
|     1|   1061|   3.0| 2.2021127|
|     1|   1343|   2.0| 2.2072542|
|     1|   2150|   3.0| 2.5715942|
|     1|   2455|   2.5| 2.1234837|
|     1|   2968|   1.0| 2.4364681|
+------+-------+------+----------+
only showing top 20 rows



In [29]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol = "rating", predictionCol="prediction")

In [30]:
rmse = evaluator.evaluate(predictDF)

In [31]:
print(rmse)

nan


In [37]:
userRecomendDF = model.recommendForAllUsers(numItems=3)

In [38]:
itemRecomendDF = model.recommendForAllItems(numUsers=3)